## Graph Neural Networks (GNNs)

### Representação de um Grafo

Matematicamente dado um grafo $G$ podemos defini-lo com um conjunto de vértices $V$ e um conjunto de arestas $E$, tal que o grafo pode ser definido com a seguinte notação: $G = (V, E)$. Cada aresta é composta por um par de vértices, representando a ligação entre eles.

<img src='./assets/example_graph.svg'/>

Nesse exemplo acima, temos como os vértices $V = (1, 2, 3, 4)$ e as arestas $E = {(1, 2), (2, 3), (2, 4), (3, 4)}$. Consideramos o grafo não direcionado, e o que isso quer dizer? Que o par de vértices, ou aresta, $(1, 2)$ é igual a $(2, 1)$.

Mas e agora, como podemos representar um grafo computacionamente? Há duas formas comumente utilizadas para representar as arestas de um grafo computacionalmente: uma matriz de adjacência ou uma lista de pares com os índices dos vértices. Enquanto para os vértices basta uma lista com seus índices e/ou propriedades.

Em aplicações os vértices e as arestas podem possuir $n$ propriedades, além de, no caso das arestas, poderem ser direcionadas. 



Uma matriz de adjacência é uma matriz quadrada com o número de linhas e colunas sendo iguais ao número de vértices. Ela informa se o vértice $i$ possui uma conexão com o vértice $j$. Sendo assim a posição $A_{ij}$ da matriz $A$ indica se os vértices $i$ e $j$ possuem alguma conexão. No caso de uma conexão entre esses vértices, a posição da matriz tem o valor $1$ atribuido, indicando essa conexão, caso contrário, é atribuído o valor $0$. Nos casos de um grafo não direcionados, a matriz $A$ será sempre uma matriz simétrica.

Para o grafo do exemplo acima temos a seguinte matriz de adjacência $A$:

$$
A = \begin{bmatrix}
    0 & 1 & 0 & 0\\
    1 & 0 & 1 & 1\\
    0 & 1 & 0 & 1\\
    0 & 1 & 1 & 0
\end{bmatrix}
$$

Enquanto expressar as arestas de um grafo por uma lista de pares de vetores é mais eficiente do ponto de vista computacional, expressar essas arestas por meio de uma matriz de adjacência pode ser mais intuitivo para humanos e mais fácil de implementar. Podemos utilizar também uma lista de arestas para definir uma matriz de adjacência esparsa com a qual podemos trabalhar como se fosse uma matriz densa, mas permitindo operações mais otimizadas em memória. O pacote `torch.sparse` possibilita trabalhar desta forma.

### Graph Convolutions

*Graph Convolutional Networks* foram introduzidas por [Kipg et al.](https://openreview.net/pdf?id=SJU4ayYgl) em 2016. Ele também escreveu um post em seu [blog](https://tkipf.github.io/graph-convolutional-networks/) sobre esse tipo de redes neurais. As GCNs são semelhantes as convoluções em imagens, uma vez que, os filtros são normalmente compartilhados por todos os locais do grafo. Da mesma forma, as GCNs contam com métodos de passagem de mensagens, o que significa que os vértices trocam informações com os vizinhos e enviam "mensagens" entre si. Antes de visualizar a matemática, podemos tentar entender como as GCNs funcionam. O primeiro passo é que cada vértice crie um vetor de recursos que representa a mensagem que deseja enviar a todos os seus vértices vizinhos. Na segunda etapa, as mensagens são enviadas aos vértices vizinhos, de forma que cada vértice receba uma mensagem para cada vizinho que possuir. Abaixo, podemos visualizar as duas etapas no grafo de exemplo.

![messagem dos vértices](./assets/graph_message_passing.svg)

Se queremos representar isso de forma matemática, primeiros precisamos decidir como combinar todas as mensagens recebidas pelos vértices. Como o número de mensagens varia ao longo de todo o grafo, precisamos de uma operação que funcione para qualquer número de mensagens. Sendo assim, uma maneira usual de realizar isso é através da soma ou da média. Dada as *features* anteriores dos vértices $H^{(l)}$, a camada GCN é definida como:

$$
H^{(l+1)} = \sigma(\hat{D}^{-1/2}\hat{A}\hat{D}^{-1/2}H^{(l)}W^{(l)})
$$

$W^{(l)}$ são os pesos com os quais transformamos nossas *features* de entrada em mensagens ($H^{(l)}W^{(l)}$). Adicionamos então a matriz de identidade a matriz de adjacência $A$, de forma que, cada vértice envie uma mensagem também para si mesmo: $\hat{A} = A + I$. Finalmente, para tirar a média ao invés de somar, calculamos a matriz $\hat{D}$, que é uma matriz diagonal com os elementos $D_{ii}$ iguais ao número de vizinhos que o vértice $i$ possui. $\sigma$ representa uma função de ativação arbitrária, e não necessariamente uma sigmoid (normalmente são utilizadas ReLU em GCNs).

Quando implementamos uma camda GCN no PyTorch, podemos utilizar as operações com tensors. Ao invés de definir uma matriz $\hat{D}$, podemos simplesmente dividir o número de mensagens pelo número de vizinhos posteriormente. Além disso, substituímos a matriz de pesos por uma camada Linear que também permite adicionar um bias. Podemos escrever um módulo GCN em PyTorch da seguinte forma:

In [1]:
import torch
from torch import nn

class GCNLayer(nn.Module):
    
    def __init__(self, c_in, c_out):
        super().__init__()
        self.projection = nn.Linear(c_in, c_out)
    
    def forward(self, node_feats, adj_matrix):
        """
        Inputs:
            node_feats - Tensor com as features do shape de um vértice (batch_size, num_nodes, c_in).
            adj_matrix - Batch de matrizes de adjacência do grafo. Se houver uma matriz de adjacência de i para j
                         adj_matriz[b,i,j] = 1 else 0. Suporta arestas direcionadas com matrizes são simétricas. Presume
                         que as conexões da matriz identidade já foram adicionadas.
                         Shape: [batch_size, num_nodes, num_nodes]
        """
        num_neighbours = adj_matrix.sum(dim=-1, keepdims=True)
        node_feats = self.projection(node_feats)
        node_feats = torch.bmm(adj_matrix, node_feats)
        node_feats = node_feats / num_neighbours
        return node_feats


Para entender melhor a camada, podemos aplicá-la ao nosso grafo de exemplo. Primeiro vamos especificar algumas *features* dos vértices e a matriz de adjacência:

In [3]:
nodes_feats = torch.arange(8, dtype=torch.float32).view(1, 4, 2)
print('Nodes features: \n{}'.format(nodes_feats))

Nodes features: 
tensor([[[0., 1.],
         [2., 3.],
         [4., 5.],
         [6., 7.]]])


In [5]:
adj_matrix = torch.Tensor([[[1, 1, 0, 0],
                             [1, 1, 1, 1],
                             [0, 1, 1, 1],
                             [0, 1, 1, 1]]])
print('Adjacency matrix: \n{}'.format(adj_matrix))

Adjacency matrix: 
tensor([[[1., 1., 0., 0.],
         [1., 1., 1., 1.],
         [0., 1., 1., 1.],
         [0., 1., 1., 1.]]])


Agora vamos aplicar uma camada GCN. Para simplificar, inicializamos a matriz linear de pesos como uma matriz identidade para que as *features* de input sejam iguais as mensagens. Isso facilita a passagem delas.

In [8]:
layer = GCNLayer(c_in=2, c_out=2)
layer.projection.weight.data = torch.Tensor([[1., 0.], [0., 1.]])
layer.projection.bias.data = torch.Tensor([0., 0.])

with torch.no_grad():
    out_feats = layer(nodes_feats, adj_matrix)

print('Adjacency matrix: \n{}'.format(adj_matrix))
print()
print('Input features: \n{}'.format(nodes_feats))
print()
print('Output features: \n{}'.format(out_feats))

Adjacency matrix: 
tensor([[[1., 1., 0., 0.],
         [1., 1., 1., 1.],
         [0., 1., 1., 1.],
         [0., 1., 1., 1.]]])

Input features: 
tensor([[[0., 1.],
         [2., 3.],
         [4., 5.],
         [6., 7.]]])

Output features: 
tensor([[[1., 2.],
         [3., 4.],
         [4., 5.],
         [4., 5.]]])


Como podemos observar, os valores de saída para o primeiro vértice são a média de si mesmo e do segundo vértice. Da mesma forma podemos verificar todos os outros vértices. No entando, em um GNN, também gostaríamos de permitir a troca de recursos entre os vértices além de seus vizinhos. Isso pode ser conseguido aplicando várias camadas de GCN, o que nos dá o layout final de uma GNN. A GNN pode ser construída por uma série de camadas GCN e não linearidades como a ReLU. Para visualização basta observar a figura abaixo (Tomas Kipf, 2016).

![Arquitetura de uma GNN](./assets/gcn_network.png)

No entanto, podemos observar o seguinte, um problema na saída é que as saídas dos vértices 3 e 4 são os mesmos já que eles possuem os mesmos vértices adjacentes. Portanto as camadas GCN podem fazer os vértices esquecer suas informações específicas se apenas tomarmos uma média sobre todas as mensagens. Várias melhorias possíveis foram propostas ao longo dos anos. Enquanto a opção mais simples é colocar conexões residuais, a abordagem mais comum é avaliar as autoconexões mais alto ou definir uma matriz de peso separada para autoconexões. Como alternativa, podemos revisitar um conceito: *attention*.

### Graph Attention

#### O que é *attention*?

O mecanismo de *attention* descreve um novo grupo de camadas de redes neurais que tem atraído bastante interesse recentemente, especialmente para tarefas de sequências. Existem muitas definições de *attention* na literatura, mas a que melhor se encaixa nesse contexto é: o mecanismo de *attention* descreve uma média ponderada de (sequência) elementos com os pesos calculados dinamicamente com base em uma *input query* e as chaves dos elementos. Então o que isso quer dizer? O Objetivo é obter uma média das características de vários elementos. No entanto, em vez de ponderar cada elemento igualmente, queremos ponderar eles dependendo de seus valores reais. Em outras palavras, queremos decidir dinamicamente quais *inputs* queremos "atender" mais do que outras. Em geral, o mecanismo de *attention* tem quatro partes que precisamos especificar:

- **Query**: A consulta é um vetor de *features* que descreve o que estamos procurando na sequência, ou seja, o que queremos prestar atenção.
- **Keys**: Para cada elemento de entrada, temos uma chave que é novamente um vetor de *feature*. Este vetor de *features* descreve aproximadamente o que o elemento está oferecendo ou quando pode ser importante. As chaves devem ser projetadas de forma que possamos identificar os elementos os quais queremos prestar atenção com base na *query*.
- **Values**: Para cada elemento de entrada, também temos um vetor de valores. Esse vetor é aquele sobre o qual queremos fazer a média.
- **Score function**: Para classificar os elementos os quais queremos prestar atenção, devemos declarar uma função de pontuação, ou, *score function*. A função de pontuação recebe a *query* e uma chave como entrada e produz a pontuação/peso de atenção do par *query*-chave. Geralmente é implementado a partir de métricas de similaridade simples , como um produto escalar ou um pequeno MLP. 

Os pesos da média são calculados por um softmax sobre todas as saídas da função de pontuação. Portanto, atribuímos a esses vetores de valor um peso maior, cuja chave correspondente é mais semelhante à consulta. Se tentamos descreve-lo com pseudo-matemática podemos escrever:

$$
\alpha_i = \frac{exp(f_{attn}(key_i, query)}{\sum{_jexp(f_{attn}(key_j, query))}}, out = \sum{\alpha_i \cdot value_i}
$$

Visualmente podemos demonstrar o *attention* sobre uma sequência de palavras da seguinte maneira:

![Demonstração do attention](./assets/attention_example.svg)

Para cada palavra temos uma chave e um valor. A consulta é comparada a todas as chaves com uma função de pontuação (neste caso, o produto escalar) para determinar os pesos. O softmax nesse caso não é visualizado a fim de simplificar o exemplo. Finalmente, os vetores de valor de todas as palavras são calculados usando pesos de *attention*.

A maioria dos mecanismos de atenção difere em termos de quais consultas eles usam, como os vetores de chave e valor são definidos e qual função de pontuação é usada.

Esse conceito pode ser aplicado de forma semelhante nos grafos, um deles é a *Graph Attention Network* (denominada GAT, proposta por [Velickovic et al., 2017](https://arxiv.org/abs/1710.10903)). Similar à GCN, a camada de atenção do grafo cria uma mensagem para cada nó usando uma camada linear/matriz de peso. Para a *attention part*, ele usa a mensagem do próprio vértice como uma consulta e as mensagens para calcular a média como chaves e valores (observe que isso também inclui a mensagem para ele mesmo). A função de pontuação $f_attn$ é implementada como um MLP de uma camada que mapeia a consulta e a chave para um único valor. O MLP tem a seguinte arquitetura (Velickovic et al.):

![Arquitetura fattn mlp](./assets/graph_attention_MLP.svg)

$h_i$ e $h_j$ são as *features* orginais dos vértices $i$ e $j$ respectivamente, e representam as mensagens da camada com o $W$ sendo a matriz de pesos. $a$ é a matriz de pesos da MLP, que tem um tamanho de $[1,2 x d_{message}]$, e $a_{ij}$ a o peso final do *attention* do vértice $i$ ao $j$. O cálculo pode ser descrito como o seguinte:

$$
a_ij = \frac{exp(LeakyReLU(a[Wh_i||Wh_j]))}{\sum_{k \in N_i}exp(LeakyReLU(a[Wh_i||Wh_k]))}
$$

O operador $||$ representa concatenação, e $N_i$ representa os índices dos vizinhos ao vértice $i$. Observe que, em contraste com a prática usual, aplicamos uma não linearidade (aqui LeakyReLU) antes do softmax sobre os elementos. Embora pareça uma pequena alteração no início, é crucial que a atenção dependa da entrada original. Especificamente, vamos remover a não linearidade por um segundo e tentar simplificar a expressão:

$$
\begin{split}\begin{split}
    \alpha_{ij} & = \frac{\exp\left(\mathbf{a}\left[\mathbf{W}h_i||\mathbf{W}h_j\right]\right)}{\sum_{k\in\mathcal{N}_i} \exp\left(\mathbf{a}\left[\mathbf{W}h_i||\mathbf{W}h_k\right]\right)}\\[5pt]
    & = \frac{\exp\left(\mathbf{a}_{:,:d/2}\mathbf{W}h_i+\mathbf{a}_{:,d/2:}\mathbf{W}h_j\right)}{\sum_{k\in\mathcal{N}_i} \exp\left(\mathbf{a}_{:,:d/2}\mathbf{W}h_i+\mathbf{a}_{:,d/2:}\mathbf{W}h_k\right)}\\[5pt]
    & = \frac{\exp\left(\mathbf{a}_{:,:d/2}\mathbf{W}h_i\right)\cdot\exp\left(\mathbf{a}_{:,d/2:}\mathbf{W}h_j\right)}{\sum_{k\in\mathcal{N}_i} \exp\left(\mathbf{a}_{:,:d/2}\mathbf{W}h_i\right)\cdot\exp\left(\mathbf{a}_{:,d/2:}\mathbf{W}h_k\right)}\\[5pt]
    & = \frac{\exp\left(\mathbf{a}_{:,d/2:}\mathbf{W}h_j\right)}{\sum_{k\in\mathcal{N}_i} \exp\left(\mathbf{a}_{:,d/2:}\mathbf{W}h_k\right)}\\
\end{split}\end{split}
$$

Podemos ver que sem a não linearidade, o termo de atenção com $h_i$ na verdade se anula, resultando na atenção sendo independente do próprio nó. Portanto, teríamos o mesmo problema que o GCN de criar os mesmos recursos de saída para nós com os mesmos vizinhos. É por isso que o LeakyReLU é crucial e adiciona alguma dependência de $h_i$ à atenção.


Depois de obter todos os fatores de *attention*, podemos calcular os recursos de saída para cada nó realizando a média ponderada:

$$
h_i'=\sigma\left(\sum_{j\in\mathcal{N}_i}\alpha_{ij}\mathbf{W}h_j\right)
$$

$\sigma$ é mais uma não linearidade, como na camada GCN. Visualmente, podemos representar a mensagem completa passando em uma camada de atenção da seguinte forma (Velickovic et al.):

![Atenção em uma GCN](./assets/graph_attention.jpeg)

Para aumentar a expressividade da Graph Attention Network, Velickovic et al. propôs estendê-lo a múltiplas *heads* similar ao bloco *Multi-Head Attention* em *Transformers*. Isso resulta em $N$ camadas de atenção sendo aplicadas em paralelo. Na imagem acima, ele é visualizado como três cores diferentes de setas (verde, azul e roxo) que são posteriormente concatenadas. A média é aplicada apenas para a camada de previsão final em uma rede.

Depois de discutir a camada de *attention* do grafo em detalhes, podemos implementá-la abaixo:

In [16]:
import torch
from torch import nn
from torch.nn import functional as F

class GATLayer(nn.Module):

    def __init__(self, c_in, c_out, num_heads=1, concat_heads=True, alpha=0.2):
        """
        Inputs:
            c_in - Dimensão das features de entrada
            c_out - Dimensão das features de saída
            num_heads - Número de heads, i.e. mecanismos de attention que serão aplicados paralelamente.
                        As features de saída são igualmente divididos entre os diferentes heads se concat_heads=True.
            concat_heads - Se True, a saída dos diferentes heads é concatenado ao ínves de ser retirado a média. 
            alpha - inclanação negativa da ativação da LeakyReLU
        """
        super().__init__()
        self.num_heads = num_heads
        self.concat_heads = concat_heads
        if self.concat_heads:
            assert c_out % num_heads == 0, "Número de outputs deve ser múltiplo do número de heads"
            c_out = c_out // num_heads

        #
        self.projection = nn.Linear(c_in, c_out * num_heads)
        self.a = nn.Parameter(torch.Tensor(num_heads, 2* c_out))
        self.leakyrelu = nn.LeakyReLU(alpha)
        
        nn.init.xavier_uniform_(self.projection.weight.data, gain=1.414)
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

    def forward(self, node_feats, adj_matrix, print_attn_probs=False):
        """
        Inputs:
            node_feats - Features de entrada do vértice. Shape: [batch_size, c_in]
            adj_matrix - Matriz de adjacência incluindo self-connections. Shape: [batch_size, num_nodes, num_nodes]
            print_attn_probs - Se True, os pesos de attention são printados durante o forward (propositos de debbuging).
        """
        batch_size, num_nodes = node_feats.size(0), node_feats.size(1)

        # Aplica a camada linear e ordena os vértices pelo head
        node_feats = self.projection(node_feats)
        node_feats = node_feats.view(batch_size, num_nodes, self.num_heads, -1)

        # Precisamos calcular o attention logits para cada aresta na nossa matriz de adjacencia
        # Fazer isso em todas as combinações possíveis é muito custoso
        # => Cria um tensor de [W*h_i||W*h_j] com i e j sendo os índices de todos os vetores
        edges = adj_matrix.nonzero(as_tuple=False) # Retorna os indices onde a matriz de adjacencia não é = 0
        node_feats_flat = node_feats.view(batch_size * num_nodes, self.num_heads, -1)
        edge_indices_row = edges[:,0] * num_nodes + edges[:,1]
        edge_indices_col = edges[:,0] * num_nodes + edges[:,2]
        a_input = torch.cat([
            torch.index_select(input=node_feats_flat, index=edge_indices_row, dim=0),
            torch.index_select(input=node_feats_flat, index=edge_indices_col, dim=0)
        ], dim=-1) # index_select retorna um tensor com node_feats_flat sendo indexado nas posições desejadas ao longo da dim=0

        # Calcula a saída de attention da MLP
        attn_logits = torch.einsum('bhc,hc->bh', a_input, self.a)
        attn_logits = self.leakyrelu(attn_logits)
        
        # Mapeia a lista de attentions de volta para uma matriz
        attn_matrix = attn_logits.new_zeros(adj_matrix.shape + (self.num_heads,)).fill_(-9e15)
        attn_matrix[adj_matrix[..., None].repeat(1, 1, 1, self.num_heads) == 1] = attn_logits.reshape(-1)
        
        # Média ponderada do attention
        attn_probs = F.softmax(attn_matrix, dim=2)
        if print_attn_probs:
            print('Attention probs: \n{}'.format(attn_probs.permute(0, 3, 1, 2)))
        node_feats = torch.einsum('bijh,bjhc->bihc', attn_probs, node_feats)
        
        if self.concat_heads:
            node_feats = node_feats.reshape(batch_size, num_nodes, -1)
        else:
            node_feats = node_feats.mean(dim=2)
        
        return node_feats


Novamente, podemos aplicar a camada de attention do grafo em nosso grafo de exemplo para entender melhor a dinâmica. Como antes, a camada de entrada é inicializada como uma matriz de identidade, mas definimos $a$ como um vetor de números arbitrários para obter diferentes valores de atenção. Usamos dois heads para mostrar os mecanismos de attention independentes e paralelos que atuam na camada.

In [19]:
layer = GATLayer(2, 2, num_heads=2)

layer.projection.weight.data = torch.Tensor([[1., 0.], [0., 1.]])
layer.projection.bias.data = torch.Tensor([0., 0.])
layer.a.data = torch.Tensor([[-0.2, 0.3], [0.1, -0.1]])

with torch.no_grad():
    out_feats = layer(nodes_feats, adj_matrix, print_attn_probs=True)

print()
print("Adjacency matrix \n{}".format(adj_matrix))
print()
print("Input features \n{}".format(nodes_feats))
print()
print("Output features \n{}".format(out_feats))

Attention probs: 
tensor([[[[0.3543, 0.6457, 0.0000, 0.0000],
          [0.1096, 0.1450, 0.2642, 0.4813],
          [0.0000, 0.1858, 0.2885, 0.5257],
          [0.0000, 0.2391, 0.2696, 0.4913]],

         [[0.5100, 0.4900, 0.0000, 0.0000],
          [0.2975, 0.2436, 0.2340, 0.2249],
          [0.0000, 0.3838, 0.3142, 0.3019],
          [0.0000, 0.4018, 0.3289, 0.2693]]]])

Adjacency matrix 
tensor([[[1., 1., 0., 0.],
         [1., 1., 1., 1.],
         [0., 1., 1., 1.],
         [0., 1., 1., 1.]]])

Input features 
tensor([[[0., 1.],
         [2., 3.],
         [4., 5.],
         [6., 7.]]])

Output features 
tensor([[[1.2913, 1.9800],
         [4.2344, 3.7725],
         [4.6798, 4.8362],
         [4.5043, 4.7351]]])


Recomendamos que você tente calcular a matriz de atenção pelo menos para um head e um vértice você mesmo. As entradas são 0 onde não existe uma aresta entre i e j. Para os outros, vemos um conjunto diversificado de probabilidades de atenção. Além disso, os recursos de saída dos nós 3 e 4 agora são diferentes, embora tenham os mesmos vizinhos.